## Part 1: Preprocessing

In [162]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import tensorflow as tf

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [163]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [164]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]



In [165]:
# Create a list of at least 10 column names to use as X data
X = ['Age', 'DistanceFromHome', 'HourlyRate', 'NumCompaniesWorked',
     'PercentSalaryHike', 'TotalWorkingYears', 'YearsAtCompany',
     'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']


# Create X_df using your selected columns
X_df = attrition_df[X]

# Show the data types for X_df
X_df.dtypes


Age                        int64
DistanceFromHome           int64
HourlyRate                 int64
NumCompaniesWorked         int64
PercentSalaryHike          int64
TotalWorkingYears          int64
YearsAtCompany             int64
YearsInCurrentRole         int64
YearsSinceLastPromotion    int64
YearsWithCurrManager       int64
dtype: object

In [166]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=13)


In [167]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
print(X_train.shape, X_test.shape)

(1102, 10) (368, 10)


In [168]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [169]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder

y_encoder = OneHotEncoder()

# Fit the encoder to the training data
y_encoded_dept = y_encoder.fit_transform(y_df[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_dept = y_encoder.transform(y_train[['Department']])
y_test_encoded_dept = y_encoder.transform(y_test[['Department']])



In [170]:
# Create a OneHotEncoder for the Attrition column
y_encoder_att = OneHotEncoder()


# Fit the encoder to the training data
y_encoded_att = y_encoder_att.fit_transform(y_df[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_att = y_encoder_att.transform(y_train[['Attrition']])
y_test_encoded_att = y_encoder_att.transform(y_test[['Attrition']])


In [171]:
# Print all the shapes
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train_encoded_dept.shape)
print(y_test_encoded_dept.shape)
print(y_train_encoded_att.shape)
print(y_test_encoded_att.shape)


(1102, 10)
(368, 10)
(1102, 3)
(368, 3)
(1102, 2)
(368, 2)


## Create, Compile, and Train the Model

In [172]:
# Find the number of columns in the X training data
X_col_len = len(X_df.columns)

# Create the input layer
input_layer = layers.Input(X_col_len, name='input')

# Create at least two shared layers
shared_layer1 = layers.Dense(units=8, activation='relu')(input_layer)
shared_layer2 = layers.Dense(units=4, activation='relu')(shared_layer1)


In [173]:
# Create a branch for Department
# with a hidden layer and an output layer
department_dense = layers.Dense(X_col_len, activation='relu')(shared_layer1)
department_output = layers.Dense(2, activation='softmax', name='department_output')(department_dense)

# Create the hidden layer
dept_hid = layers.Dense(8, activation='relu')(department_output)

# Create the output layer
dept_out = layers.Dense(3, activation='softmax', name='dept_out')(dept_hid)


In [174]:
# Create a branch for Attrition
# with a hidden layer and an output layer
att_dense = layers.Dense(X_col_len, activation='relu')(shared_layer1)
att_output = layers.Dense(2, activation='softmax', name='att_output')(att_dense)

# Create the hidden layer
att_hid = layers.Dense(8, activation='relu')(att_output)

# Create the output layer
att_out = layers.Dense(2, activation='softmax', name='att_out')(att_hid)


In [175]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_out, att_out])

# Compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Summarize the model
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 10)]                 0         []                            
                                                                                                  
 dense_50 (Dense)            (None, 8)                    88        ['input[0][0]']               
                                                                                                  
 dense_52 (Dense)            (None, 10)                   90        ['dense_50[0][0]']            
                                                                                                  
 dense_54 (Dense)            (None, 10)                   90        ['dense_50[0][0]']            
                                                                                            

In [176]:
# Train the model
def train(model, X_train_scaled, y_train_encoded_dept, y_train_encoded_att):
    y_train_encoded_dept = y_train_encoded_dept.toarray()
    y_train_encoded_att = y_train_encoded_att.toarray()
    model.fit(X_train_scaled, {'dept_out': y_train_encoded_dept, 'att_out': y_train_encoded_att}, epochs=100)
    return model
train(model, X_train_scaled, y_train_encoded_dept, y_train_encoded_att)


Epoch 1/100


35/35 [==============================] - 1s 857us/step - loss: 1.3870 - dept_out_loss: 0.7115 - att_out_loss: 0.6755 - dept_out_accuracy: 0.3158 - att_out_accuracy: 0.7686
Epoch 2/100
35/35 [==============================] - 0s 792us/step - loss: 1.2862 - dept_out_loss: 0.6392 - att_out_loss: 0.6471 - dept_out_accuracy: 0.3258 - att_out_accuracy: 0.8376
Epoch 3/100
35/35 [==============================] - 0s 779us/step - loss: 1.2020 - dept_out_loss: 0.5885 - att_out_loss: 0.6134 - dept_out_accuracy: 0.5762 - att_out_accuracy: 0.8376
Epoch 4/100
35/35 [==============================] - 0s 801us/step - loss: 1.1331 - dept_out_loss: 0.5559 - att_out_loss: 0.5772 - dept_out_accuracy: 0.6343 - att_out_accuracy: 0.8376
Epoch 5/100
35/35 [==============================] - 0s 819us/step - loss: 1.0784 - dept_out_loss: 0.5341 - att_out_loss: 0.5443 - dept_out_accuracy: 0.6343 - att_out_accuracy: 0.8376
Epoch 6/100
35/35 [==============================] - 0s 796us/step - loss: 1.0352 - dept_out

In [179]:
# Evaluate the model with the testing data
model.evaluate(X_test_scaled, {'dept_out': y_test_encoded_dept.toarray(), 'att_out': y_test_encoded_att.toarray()})

12/12 [==============================] - 0s 819us/step - loss: 0.8568 - dept_out_loss: 0.4377 - att_out_loss: 0.4190 - dept_out_accuracy: 0.7120 - att_out_accuracy: 0.8288


[0.8567848205566406,
 0.4377400875091553,
 0.4190447926521301,
 0.7119565010070801,
 0.8288043737411499]

In [178]:
# Print the accuracy for both department and attrition
dept_acc, att_acc = model.evaluate(X_test_scaled, {'dept_out': y_test_encoded_dept.toarray(), 'att_out': y_test_encoded_att.toarray()})[3:]
print(f"Department Accuracy: {dept_acc}")
print(f"Attrition Accuracy: {att_acc}")

12/12 [==============================] - 0s 956us/step - loss: 0.8568 - dept_out_loss: 0.4377 - att_out_loss: 0.4190 - dept_out_accuracy: 0.7120 - att_out_accuracy: 0.8288
Department Accuracy: 0.7119565010070801
Attrition Accuracy: 0.8288043737411499


In [181]:
print(y_df['Department'].value_counts(),
y_df['Attrition'].value_counts())

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64 Attrition
No     1233
Yes     237
Name: count, dtype: int64


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Yes because both classes are the same size.
2. I chose softmax because I had a better accuracy as opposed to sigmoid, but that
makes sense as this is not binary classification like I first thought.
3. -Less diverse data within departments, and attrition
   
   